In [1]:
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import requests
import re
import time 

ModuleNotFoundError: No module named 'selenium'

In [ ]:
#url for site
url = "https://www.starbucks.com/store-locator?map=36.15032,-86.735135,9z&place=Davidson%20County,%20TN,%20USA"

#defining location of chrome driver in files
chrome_driver = "C:\Program Files (x86)\chromedriver.exe"

#creating browser pop up 
browser = webdriver.Chrome(chrome_driver)
#loading website in browser
browser.get(url)
time.sleep(3)
#click agree button for cookies pop up
click_agree = browser.find_element(By.CSS_SELECTOR, ("#truste-consent-button")).click();
#sleep so page loads after agressing to cookies
time.sleep(5)

html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

In [ ]:
#checking soup works
soup.find('title').text

In [ ]:
#find location names in soup
info_loc = soup.findAll('h3', attrs = {'class' : 'sb-heading text-xxs text-bold'})
info_loc
#create list to store location names
#loc_names = {}
#looking in info_loc and pulling out location name and putting into list
#for info in info_loc:
   # loc_names = info.get_text()
  #  print(loc_names)
    

In [ ]:
#create list to store location names
loc_names = [loc.text for loc in info_loc]
loc_names[0]

In [ ]:
address_info = soup.findAll('p', attrs = {'data-e2e' : "address"})
address_info

In [ ]:
#create list to store location names
loc_address = [loc.text for loc in address_info]
loc_address

In [ ]:
starbucks = pd.DataFrame()
starbucks['location'] = loc_names
starbucks['address'] = loc_address
starbucks['address_new'] = starbucks['address'] + ',TN'
starbucks.head()

In [ ]:
starbucks.to_csv('starbucks_locs.csv')

In [ ]:
#list comprehension 
xpaths = ", ".join([f"//a[@aria-label='Store details for {loc}']//*[name()='svg']" for loc in loc_names])
xpaths

In [ ]:
xpaths_list = xpaths.split(", ")
type(xpaths_list)

In [ ]:
len(xpaths_list)

In [ ]:
#info = browser.find_element(By.XPATH, '//a[@aria-label="Store details for Nipper\'s Corner"]').click();
#time.sleep(3)

In [ ]:

#current_page = browser.page_source
#response = requests.get(url2)
#soup2 = BeautifulSoup(current_page)
#soup2

In [ ]:
#container = soup2.find('section', attrs = {'class' : 'pb5 px4 lg-px6'})
#container

In [ ]:
#for paths in xpaths_list:
    #info_button = (f"browser.find_element(By.XPATH, \"{paths}\")")
    #info_button.click()
    #time.sleep(3)
    #scrape page for html  
    #names = find_element(By.CSS_SELECTOR, "h1.sb-heading text-lg text-bold pr7 mb1")

In [ ]:
#fancy override
for index,paths in enumerate(xpaths_list):
    if paths.find("Nipper") != -1:
        xpaths_list[index] = '//a[@aria-label="Store details for Nipper\'s Corner"]'

In [ ]:
#loc_name = []
#address = []
#zipcode = []
big_soup = []

for paths in xpaths_list:
    try:
        info_button = browser.find_element(By.XPATH, paths)
        info_button.click();
        time.sleep(7)
        #get page source 
        current_page = browser.page_source
        #get soup of page
        soup2 = BeautifulSoup(current_page)
        #find container in soup and place in list
        container_soup = soup2.find('section', attrs = {'class' : 'pb5 px4 lg-px6'})
        big_soup.append(container_soup)
        time.sleep(7)
        #click exit 
        click_exit = browser.find_element(By.XPATH, "//button[@aria-label='Close']//*[name()='svg']").click();
        time.sleep(7)
    except: 
        print(f'something went wrong at path {paths}') 
        continue
    


In [ ]:
big_soup

In [ ]:
loc_name = big_soup[0].findAll('h1', attrs = {'class' : "sb-heading text-lg text-bold pr7 mb1"})
loc_name
loc_street = big_soup[0].findAll('span', attrs = {'class' : "block"})
loc_street[1]


In [ ]:
#pull out information
loc_names = []
loc_address = []


for container in big_soup:
    names = container.findAll('h1', attrs = {'class': "sb-heading text-lg text-bold pr7 mb1"})
    loc_names.append(names)
    address = container.findAll('span', attrs = {'class' : "block"})
    loc_address.append(address)
    
loc_address

    

In [ ]:
len(loc_address[0])

In [ ]:
zips = []

for lst in loc_address:
    if len(lst) == 2:
        z = lst[1]
        zips.append(z)
    elif len(lst) == 3:
        z = lst[2]
        zips.append(z)

zips

In [ ]:
starbucks_zips = pd.read_csv('./data/starbucks_almost.csv')
starbucks_zips['zipcodes'] = zips
starbucks_zips['zipcodes'] = starbucks_zips['zipcodes'].astype(str)
starbucks_zips['zipcodes'] = starbucks_zips['zipcodes'].str.extract('(\d{5})')

In [ ]:
starbucks_zips.to_csv('star_zips.csv')